# Gen features

In [1]:
import pandas as pd
import numpy as np
import copy

import warnings
warnings.filterwarnings('ignore')

# Read data

In [2]:
af = 'u'
dt = 't'
path = f'./data/{dt}.csv'
# path = f'./data/{dt}_sm.csv'
df = pd.read_csv(path)

In [3]:
df.head()

,StreamID,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label
0,1,2022-07-25 23:29:01.159088,442.169202,tcp,150.35.87.62,49972,<?>,92.236.177.28,443,RPA_FPA,0.0,0.0,435,371513,107279,flow=Background-Established-cmpgw-CVUT
1,2,2022-07-25 23:29:01.499175,2365.877764,udp,150.35.87.62,54863,<->,93.35.149.223,7452,CON,0.0,0.0,4,269,146,flow=Background-Established-cmpgw-CVUT
2,3,2022-07-25 23:29:01.500936,0.201912,tcp,150.35.87.62,49909,<?>,168.196.248.87,80,FA_FA,0.0,0.0,2,196,131,flow=Background-Established-cmpgw-CVUT
3,4,2022-07-25 23:29:01.502503,0.005431,tcp,150.35.87.62,49906,<?>,198.116.235.78,80,FA_FA,0.0,0.0,3,200,132,flow=Background-Established-cmpgw-CVUT
4,5,2022-07-25 23:29:01.504320,0.012403,tcp,150.35.87.62,49907,<?>,196.173.143.73,80,FA_FA,0.0,0.0,3,197,131,flow=Background-Established-cmpgw-CVUT


In [4]:
# df.loc[(df['sTos'] == 192)]
# # df.loc[(df['sTos'] == 192) & (df['Label'].str.contains('Bot'))]
# df.loc[(df['sTos'] != 192) & (df['Proto'] == 'igmp')]

# Label

In [5]:
df['LabelStr'] = df['Label']
df['Label'] = 0
df.loc[df['LabelStr'].str.contains('To-Backgro'), 'Label'] = 1
df.loc[df['LabelStr'].str.contains('From-Backg'), 'Label'] = 2
df.loc[df['LabelStr'].str.contains('From-Norma'), 'Label'] = 3
df.loc[df['LabelStr'].str.contains('To-Normal-'), 'Label'] = 4
df.loc[df['LabelStr'].str.contains('From-Botne'), 'Label'] = 5
df.loc[df['LabelStr'].str.contains('Normal-'), 'Label'] = 3

# Calculate

In [6]:
#? packets per second
df['PktsPerSec'] = df['TotPkts']/df['Dur'].replace(0, np.inf)

#? bytes (sent both directions) per second
df['BytesPerSec'] = df['TotBytes']/df['Dur'].replace(0, np.inf)

#? bytes (from src -> dst) per second
df['SrcBytesPerSec'] = df['SrcBytes']/df['Dur'].replace(0, np.inf)

#? bytes per packet
df['BytesPerPkt'] = df['TotBytes']/df['TotPkts'].replace(0, np.inf)


#? dst bytes
df['DstBytes'] = df['TotBytes'] - df['SrcBytes']
#? bytes (from dst -> src) per second
df['DstBytesPerSec'] = df['DstBytes']/(df['Dur'].replace(0, np.inf))
#? if dst bytes < 0
df.loc[df['DstBytes'] < 0, 'DstBytes'] = 0
df.loc[df['DstBytesPerSec'] < 0, 'DstBytesPerSec'] = 0

# State

In [7]:
""" some records have NaN state value """
df.loc[df['State'].isnull(), 'State'] = 'nanvalue'

In [8]:
""" some state is just to indicate the tcp states, replace these states with alltcp and use Flag_ fields to indicate triggered flags """
df['State_orig'] = df['State']
df.loc[(df['State_orig'].str.len() > 2) & (df['State_orig'].str.contains('_')), 'State'] = 'alltcp'

# sTos & dTos

In [9]:
""" Some sTos and dTos has nan value """
df.loc[df['sTos'].isnull(), 'sTos'] = -1
df.loc[df['dTos'].isnull(), 'dTos'] = -1

df = df.drop(df[df['sTos'] == 192.0].index)
df = df.drop(df[df['dTos'] == 192.0].index)

In [10]:
print(df['sTos'].value_counts())
print()
print(df['dTos'].value_counts())
print()
# df.loc[df['sTos'] == 192]

 0.0    936649
-1.0      1155
 2.0         3
 3.0         2
Name: sTos, dtype: int64

 0.0    885711
-1.0     52070
 2.0        15
 3.0        11
 1.0         2
Name: dTos, dtype: int64



# Save the processed data

In [11]:
df.to_csv(path.replace('.csv', f'.{af}1.csv'))

In [12]:
del df